In [2]:
%load_ext autoreload
%autoreload 2
from replicaEVSE import datautils as du
import os
import dask.dataframe as dd
import pandas as pd

datadir = '../../data/'

In [12]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8)  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
client

/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60961 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:60961/status,
Dashboard: http://127.0.0.1:60961/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60962,Workers: 8
Dashboard: http://127.0.0.1:60961/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:60994,Total threads: 1
Dashboard: http://127.0.0.1:60996/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:60965,


### load in the trips and pop data and the join of the two for desitinations that end in WA

In [7]:
pop_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_population.parquet')
trip_thu_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_thursday_trip.parquet')
merged_ddf = dd.read_parquet(datadir+'/wa_pop_and_trips_sorted.parquet') # len = 51727268

### Ubers

In [10]:
ubers = trip_thu_ddf.loc[trip_thu_ddf['mode'] == 'ON_DEMAND_AUTO']
ubers['person_id'].value_counts().compute()

990158709039785955      16
9183962885016971228     15
7236881691111520416     14
17416661576243475932    11
14826177188316444853    11
                        ..
16069901924244521885     1
16069935096932885818     1
16069955311810168656     1
1607009112529347857      1
9999937766891715451      1
Name: person_id, Length: 310740, dtype: int64

### Trucks
only thursday for now

In [13]:
trucks = trip_thu_ddf.loc[trip_thu_ddf['mode'] == 'COMMERCIAL'] # len = 2190600
trucks['person_id'].value_counts().compute()

10000011262318393978    1
4466000335956290558     1
446616179920110017      1
4466161462016108901     1
4466155850483027306     1
                       ..
15536171076438081648    1
15536163747924793994    1
15536162389987112307    1
155361606456951021      1
9999993057755165125     1
Name: person_id, Length: 2190600, dtype: int64

There doesn't seem to be any persons associated with commercial in the population data

Confirmed: No persons in trucking in the population dataset

In [26]:
truck_person_list = list(trucks['person_id'])
pop_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_population.parquet')
out = pop_ddf[pop_ddf['person_id'].isin(truck_person_list)]
# len(out) = 0
